## Magic Methods and the Python data model

We've already seen 3 magic methods: `__init__`, `__repr__`, and `__str__`

It turns out that we can get a lot of power by providing implementations of a few more magic methods to make our classes behave like other types (arithmetic types collection types, mainly).

(For a list of all the magic methods you can override, see https://docs.python.org/3/reference/datamodel.html)

Let's consider a Pythonic deck of cards:


In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class Deck:
    # ranks and suits are class attributes because they
    # should be shared by all decks
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
#         self._cards = []
#         for suit in self.suits:
#             for rank in self.ranks:
#                 self._cards.append(Card(rank, suit))
        self._cards = [
            Card(rank, suit) 
            for suit in self.suits
            for rank in self.ranks
        ]

In [2]:
d = Deck()

In [3]:
# We can create a deck of cards, but it turns out it's not iterable...

for card in d:
    print(card)

TypeError: 'Deck' object is not iterable

In [4]:
# ...unless we refer to `_cards` directly
for card in d._cards:
    print(card, end=' ')

Card(rank='2', suit='clubs') Card(rank='3', suit='clubs') Card(rank='4', suit='clubs') Card(rank='5', suit='clubs') Card(rank='6', suit='clubs') Card(rank='7', suit='clubs') Card(rank='8', suit='clubs') Card(rank='9', suit='clubs') Card(rank='10', suit='clubs') Card(rank='J', suit='clubs') Card(rank='Q', suit='clubs') Card(rank='K', suit='clubs') Card(rank='A', suit='clubs') Card(rank='2', suit='diamonds') Card(rank='3', suit='diamonds') Card(rank='4', suit='diamonds') Card(rank='5', suit='diamonds') Card(rank='6', suit='diamonds') Card(rank='7', suit='diamonds') Card(rank='8', suit='diamonds') Card(rank='9', suit='diamonds') Card(rank='10', suit='diamonds') Card(rank='J', suit='diamonds') Card(rank='Q', suit='diamonds') Card(rank='K', suit='diamonds') Card(rank='A', suit='diamonds') Card(rank='2', suit='hearts') Card(rank='3', suit='hearts') Card(rank='4', suit='hearts') Card(rank='5', suit='hearts') Card(rank='6', suit='hearts') Card(rank='7', suit='hearts') Card(rank='8', suit='hear

In [5]:
# we also cannot find the length of the deck
print(len(d))

TypeError: object of type 'Deck' has no len()

In [6]:
# ...at least not without referring to `_cards` directly
print(len(d._cards))

52


## Making our deck iterable
* the Python data model allows us to accomplish quite a bit, just by implement the `__len__()` and `__getitem__()` methods

```python
lst[5]  # lst.__getitem__(5)
```

In [7]:
# a deck of cards, round two
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class Deck:
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
    def __len__(self):
        return len(self._cards)   # return self._cards.__len__()

    def __getitem__(self, position):
        # implements self[position]
        return self._cards[position]
        # return self._cards.__getitem__(position)

In [8]:
deck = Deck()
len(deck)

52

In [9]:
for card in deck:
    print(card, end=' ')

Card(rank='2', suit='clubs') Card(rank='3', suit='clubs') Card(rank='4', suit='clubs') Card(rank='5', suit='clubs') Card(rank='6', suit='clubs') Card(rank='7', suit='clubs') Card(rank='8', suit='clubs') Card(rank='9', suit='clubs') Card(rank='10', suit='clubs') Card(rank='J', suit='clubs') Card(rank='Q', suit='clubs') Card(rank='K', suit='clubs') Card(rank='A', suit='clubs') Card(rank='2', suit='diamonds') Card(rank='3', suit='diamonds') Card(rank='4', suit='diamonds') Card(rank='5', suit='diamonds') Card(rank='6', suit='diamonds') Card(rank='7', suit='diamonds') Card(rank='8', suit='diamonds') Card(rank='9', suit='diamonds') Card(rank='10', suit='diamonds') Card(rank='J', suit='diamonds') Card(rank='Q', suit='diamonds') Card(rank='K', suit='diamonds') Card(rank='A', suit='diamonds') Card(rank='2', suit='hearts') Card(rank='3', suit='hearts') Card(rank='4', suit='hearts') Card(rank='5', suit='hearts') Card(rank='6', suit='hearts') Card(rank='7', suit='hearts') Card(rank='8', suit='hear

### ...but just by implementing \_\_`getitem`\_\_`()`, we get so much more!

In [10]:
# like indexing
deck[0], deck[-1]  # deck[len(deck) - 1]

(Card(rank='2', suit='clubs'), Card(rank='A', suit='spades'))

In [11]:
# ...and slicing!
deck[9:13]   # => deck._cards[9:13]

[Card(rank='J', suit='clubs'),
 Card(rank='Q', suit='clubs'),
 Card(rank='K', suit='clubs'),
 Card(rank='A', suit='clubs')]

In [12]:
deck[12::13]

[Card(rank='A', suit='clubs'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='hearts'),
 Card(rank='A', suit='spades')]

## What about a method to pick a random card?
* no need because Python already has a function to choose a random item from a sequence

In [13]:
from random import choice

help(choice)

Help on method choice in module random:

choice(seq) method of random.Random instance
    Choose a random element from a non-empty sequence.



In [17]:
choice(deck)

Card(rank='5', suit='hearts')

In [18]:
# Jupyter magic
choice??

## Two big advantages of using special methods to leverage the Python data model
*  users of your classes don’t have to memorize arbitrary method names for standard operations (“How to get the number of items? Is it __`.size()`__, __`.length()`__, or what?”)
* it’s easier to benefit from the rich Python standard library and avoid reinventing the wheel, e.g., __`random.choice()`__

In [43]:
class Deck():
    ranks = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
    suits = 'clubs diamonds hearts spades'.split()

    def __init__(self):
        self._cards = [
            Card(rank, suit) 
            for suit in self.suits                                        
            for rank in self.ranks
        ]
        
    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        # implements self[position]
        return self._cards[position]
    
    def __setitem__(self, position, value):
        # implements self[position] = value  ==> self.__setitem__(position, value)
        self._cards[position] = value


In [40]:
# Also del deck[5] => deck.__delitem__(5)

In [41]:
import random
deck = Deck()

In [42]:
random.shuffle(deck)

list(deck)

[Card(rank='4', suit='diamonds'),
 Card(rank='K', suit='clubs'),
 Card(rank='J', suit='diamonds'),
 Card(rank='2', suit='spades'),
 Card(rank='10', suit='hearts'),
 Card(rank='4', suit='clubs'),
 Card(rank='10', suit='clubs'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='3', suit='clubs'),
 Card(rank='J', suit='hearts'),
 Card(rank='K', suit='diamonds'),
 Card(rank='7', suit='diamonds'),
 Card(rank='8', suit='diamonds'),
 Card(rank='9', suit='clubs'),
 Card(rank='5', suit='hearts'),
 Card(rank='8', suit='spades'),
 Card(rank='5', suit='spades'),
 Card(rank='6', suit='clubs'),
 Card(rank='A', suit='diamonds'),
 Card(rank='9', suit='hearts'),
 Card(rank='7', suit='spades'),
 Card(rank='8', suit='hearts'),
 Card(rank='4', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='10', suit='spades'),
 Card(rank='A', suit='clubs'),
 Card(rank='Q', suit='hearts'),
 Card(rank='A', suit='hearts'),
 Card(rank='4', suit='hearts'),
 Card(rank='6', suit='hearts'),
 Card(rank='8', suit='clubs'),

In [22]:
# also jupyter magic
random.shuffle??

# Attribute access

The magic method `__getattr__` is called by Python *whenever there would otherwise be an `AttributeError` raised*:

In [23]:
class A:
    def __getattr__(self, name):
        print(f'Calling __getattr__({name})')
        return None

In [24]:
a = A()
a.foo = 'bar'

In [25]:
a.foo  # `__getattr__` not called

'bar'

In [26]:
print(a.bat_anything_else)

Calling __getattr__(bat_anything_else)
None


In [27]:
a.__dict__

{'foo': 'bar'}

In [28]:
class Proxy:
    def __init__(self, real):
        self._real = real
        
    def __getattr__(self, name):
        print('__getattr__', name)
        if name.startswith('_'):
            raise AttributeError
        return getattr(self._real, name)

In [29]:
lst = []
p = Proxy(lst)

In [30]:
p.append('5')
# _tmp = getattr(p, 'append')
# _tmp('5')

__getattr__ append


In [31]:
p.append

__getattr__ append


<function list.append(object, /)>

In [32]:
lst

['5']

Magic methods do not get forwarded

In [33]:
repr(p)  # p._real.__repr__ is not looked up

'<__main__.Proxy object at 0x7fb04df902b0>'

In [34]:
repr(p._real)

"['5']"

# Lab

Open the [OOP Magic Lab][oop-magic-lab]

[oop-magic-lab]: ./oop-magic-lab.ipynb